In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Twitter
twitter = Twitter()

C:\Users\sbg02\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


### 데이터 불러오기

In [3]:
df = pd.read_csv('C:/newsdata/삼성전자.csv',  encoding = 'utf-8 sig', engine = 'python')
df.head()

,TITLE,BODY,TIME
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018.12.181953
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018.12.181943
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018.12.181830
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,&2018.12.181745
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,12018.12.181710


# 데이터 전처리

### TIME클렌징1 - TIME칼럼 EDA
1. for문 => 너무 복잡 & 시간이 필요<br>
<코드>  ???
2. RE => str으로 변환 뒤 '12018.' => '2018.'  으로 바꿔야 하는데 결과값이 Series가 아니라 \t처리된 str으로 나온다.<br>
<코드> pre3 = re.sub('[1\d]{4}8\.', '2018.', str(pre1))

### (아래에서 끌어온 결과)
#### 데이터 3등분 
<코드> time['year'].unique()

=> 결과<br>
array(['2018', '12018', '-2018', '', '--2018', '3002018', '22018',
       '+2018', '122018', '2017', '12017', '-2017', '22017', '112017',
       '122017']<br><br>
=> 뒤 숫자 4개만 남기면 정리될 듯?

### 연도 클렌징 - 특수문자 &클렌징    (뒤에 더 좋은 방법이 있어서 불필요할듯...)

In [4]:
pre1 = pd.Series(df['TIME']).str.replace('&', '')
pre1.head()

0     2018.12.181953
1     2018.12.181943
2     2018.12.181830
3     2018.12.181745
4    12018.12.181710
Name: TIME, dtype: object

### 연도 클렌징 - '.' 기준으로 칼럼 3등분 나누어 접근

In [5]:
time = pd.DataFrame(pre1.str.split('.',2).tolist(), columns = ['year','month','time'])
time.head()

,year,month,time
0,2018,12,181953
1,2018,12,181943
2,2018,12,181830
3,2018,12,181745
4,12018,12,181710


In [6]:
'''
https://www.geeksforgeeks.org/python-pandas-series-str-slice/
year_clean = time.year[-4:]

for i in range(0,len(df)):
    df.iloc[i].col = df.iloc[i].col[:9]
    
df.col = df.col.str[:9]   
    '''

'\nhttps://www.geeksforgeeks.org/python-pandas-series-str-slice/\nyear_clean = time.year[-4:]\n\nfor i in range(0,len(df)):\n    df.iloc[i].col = df.iloc[i].col[:9]\n    \ndf.col = df.col.str[:9]   \n    '

### 연도 클렌징(실패) - 슬라이싱한 부분을 삭제하는 코드....ㅠ

In [7]:
'''
# start stop and step variables 
start, stop, step = 0, -4, 1
  
# converting to string data type 
data["Salary"]= data["Salary"].astype(str) 
  
# slicing till 4rd last element 
time['year'] = time['year'].str.slice(start, stop, step) 
  
# display 
time.head()
'''

'\n# start stop and step variables \nstart, stop, step = 0, -4, 1\n  \n# converting to string data type \ndata["Salary"]= data["Salary"].astype(str) \n  \n# slicing till 4rd last element \ntime[\'year\'] = time[\'year\'].str.slice(start, stop, step) \n  \n# display \ntime.head()\n'

### 연도 클렌징(성공) - 람다 사용해 단순 백슬라이싱

In [8]:
time['year'] = time['year'].apply(lambda x: x[-4:])
time['year'].unique()

array(['2018', '', '2017'], dtype=object)

### 시간 클렌징 - 초 삭제

In [9]:
# start stop and step variables 
start, stop, step = 0, -2, 1
  
# slicing till 2nd last element 
time['time'] = time['time'].str.slice(start, stop, step) 
  
# display 
time.head()

,year,month,time
0,2018,12,1819
1,2018,12,1819
2,2018,12,1818
3,2018,12,1817
4,2018,12,1817


### 마무리 - 원본에 붙이기

In [10]:
# 칼럼 삭제
del df['TIME']

In [11]:
# 칼럼 합치기
df_timeok = pd.concat([df, time], axis = 1, sort = False)
df_timeok.head()

,TITLE,BODY,year,month,time
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018,12,1819
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018,12,1819
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018,12,1818
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,2018,12,1817
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,2018,12,1817


# 시간정제 끝!!   -결과 = df

In [12]:
df = df_timeok

In [13]:
df.head()

,TITLE,BODY,year,month,time
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018,12,1819
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018,12,1819
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018,12,1818
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,2018,12,1817
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,2018,12,1817


### sample데이터만으로 pilot분석
### 쉽게 해볼까?

In [ ]:
df.head()

In [ ]:
# 퍼포먼스 위해 일단 5개 data만 사용
dfdf = df.head()
dfdf

In [ ]:
# title과 body부분을 한방에 먼저 합쳐버림
dfdf['ab'] = dfdf.apply(lambda x : '{}|||{}'.format(x['TITLE'], x['BODY']), axis = 1)

In [ ]:
dfdf

In [ ]:
dfdf['ab'] = dfdf['ab'].astype(str)
dfdf['abc'] = dfdf['ab'].apply(lambda x : twitter.nouns(x))

In [ ]:
# 정상적으로 분석됐는지 확인 => 결과 양호
dfdf.loc[0,'abc']

### pilot분석 성공!!

# 형태소 분석 - 명사 추출
### pilot 성공 후 본겜!!

In [14]:
# title과 body 합쳐서 새로운 칼럼 추가  구분자 : |||
df['title_body_sum'] = df.apply(lambda x : '{}|||{}'.format(x['TITLE'], x['BODY']), axis = 1)

In [15]:
# 확인
df.head()
#df.loc[1, 'title_body_sum']

,TITLE,BODY,year,month,time,title_body_sum
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018,12,1819,삼성전자 강경훈 부사장 19일 구속심사|||[theL]삼성에버랜드 노조 설립 방해·...
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018,12,1819,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구|||[the L]'삼성...
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018,12,1818,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결|||[특집]삼성전자는..."
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,2018,12,1817,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상|||인탑스(주)..."
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,2018,12,1817,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사|||염호석씨 시신탈취 과정...


### 아래는 형태소 분석기 돌리는 코드
#### 시간이 오래걸립니다!!!

In [16]:
# 분석할 칼럼을 str으로 변환 확인 후 분석기 돌리기 (가끔 오류가 납니다...)
df['title_body_sum'] = df['title_body_sum'].astype(str)
df['NOUNS'] = df['title_body_sum'].apply(lambda x : twitter.nouns(x))

In [17]:
df.head()

,TITLE,BODY,year,month,time,title_body_sum,NOUNS
0,삼성전자 강경훈 부사장 19일 구속심사,[theL]삼성에버랜드 노조 설립 방해·노조 와해 혐의에버랜드 노조 와해 의혹을 수...,2018,12,1819,삼성전자 강경훈 부사장 19일 구속심사|||[theL]삼성에버랜드 노조 설립 방해·...,"[삼성, 전자, 강경, 훈, 부사, 구속, 심사, 삼성, 에버랜드, 노조, 설립, ..."
1,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구,[the L]'삼성 노조 와해 의혹'을 수사 중인 검찰이 강경훈 삼성전자 부사장에 ...,2018,12,1819,檢 '노조 와해 의혹' 강경훈 삼성전자 부사장 구속영장 청구|||[the L]'삼성...,"[노조, 의혹, 강경, 훈, 삼성, 전자, 부사, 구속영장, 청구, 삼성, 노조, ..."
2,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결",[특집]삼성전자는 1995년 국내 기업 최초로 사회봉사단을 창단 사회문제 해결을 위...,2018,12,1818,"""사회가 건강해야 기업도 발전"" 20년 넘은 삼성 나눔의 비결|||[특집]삼성전자는...","[사회, 기업, 발전, 삼성, 나눔, 비결, 특집, 전자, 국내, 기업, 최초, 사..."
3,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상",인탑스(주)(대표 김근하 정사진)가 스마트팩토리 도입을 추진해 경북창조경제혁신센터가...,2018,12,1817,"4차 산업 혁신기업 인탑스, 스마트팩토리 중소벤처기업부 장관상 수상|||인탑스(주)...","[차, 산업, 혁신, 기업, 인탑스, 스마트, 팩토리, 중소, 벤처기업, 부, 장관..."
4,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사,염호석씨 시신탈취 과정서 뇌물수수 경찰관도 함께(서울=뉴스1) 나연준 기자 = 삼성...,2018,12,1817,'에버랜드 노조와해' 삼성 노사총괄 부사장 내일 영장심사|||염호석씨 시신탈취 과정...,"[에버랜드, 노조, 해, 삼성, 노사, 총괄, 부사, 내일, 영장, 심사, 염호, ..."


In [20]:
#df.loc[1, 'NOUNS']

# =============================================================

# 형태소 분석 삽질Part..... (공부용으로 사용)

In [ ]:
# 제목에서 명사만 추출해 a라는 변수에 list형태로 저장
# df['TITLE'].apply(lambda x : twitter.nouns(x))

df['TITLE'] = df['TITLE'].astype(str)
df['a'] = df['TITLE'].apply(lambda x : twitter.nouns(x))

In [ ]:
# 오류때문에 본문을 str타입으로 변환하는 작업을 진행
# 본문에서 명사만 추출해 b라는 변수에 list로 저장

df['BODY'] = df['BODY'].astype(str)
df['b'] = df['BODY'].apply(lambda x : twitter.nouns(x))

In [ ]:
dfdf

### title과 body의 데이터 합쳐서 원본df 옆에 새 칼럼 추가하기
* 방법 1 : Series 형태로 나온걸 합친 뒤 df에 추가
* 방법 2 : 각각 df에 추가한 뒤 두 칼럼 하나로 합치기
    * => 방법 1이 더 퍼포먼스가 좋을 듯??
* 아니면 애초에 title과 body를 합치고 분석기 돌리는게 나을수도...    =>    위에서 결국 쉽게 풀어버림...

퍼포먼스를 고려한 합치기 timespend (멘토님말대로 람다가 최고)
https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python

### 방법2

In [ ]:
# 뽑아낸 데이터 만지기 위해 축소
aa = a[:5]
bb = b[:5]
# 요렇게 하고 이것저것 실험

In [ ]:
# 원본 데이터 프레임에 임시로 추가
df['a'] = a
df['b'] = b
df.tail()

In [ ]:
dfdf = df.head()

In [ ]:
dfdf

In [ ]:
# title과 body의 명사를 합쳐서 하나의 칼럼으로 만들기


In [ ]:
# 파워풀 람다 합치기
dfdf['ab'] = dfdf.apply(lambda x : '{}{}'.format(x['a'], x['b']), axis = 1)

In [ ]:
# 합쳐지긴 했는데 칼럼에 list가 두개가 있다.
dfdf.loc[1, 'ab']

In [ ]:
dfdf['ab'] = dfdf[['a', 'b']].apply(lambda x :''.join(x), axis = 1)
#df['period'] = df[['Year', 'quarter']].apply(lambda x: ''.join(x), axis=1)

### 리스트 붙이는 TEST
a1 = [1,2,3]<br>
b1 = [9,8,7]<br><br>

a1.append(b1)<br>
print(a1)<br><br>
a1.extend(b1)<br>
print(a1)<br>
> append()는 원본 리스트 '속'에다 새 리스트를 집어넣는 함수
> extend()는 원본 리스트 옆에 새 리스트를 나란히 붙이는 함수

In [ ]:
# 중복데이터 제거 후 합쳐주는 함수 (list set)
df['ids'] = df['ids'].apply(lambda x: list(set(x)))

# 두 시리즈에서 조건에 맞는 것만 골라 가져오는 함수
s1.combine(s2, lambda x1, x2: x1 if x1 < x2 else x2)

In [ ]:
df['name'] = df['name'].apply(lambda e: e.split()[0])

>>> from timeit import Timer
>>> t = {}
>>> t['methodcaller'] = Timer("map(methodcaller('split', ' '), a)", "from operator import methodcaller; a=['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> t['lambda'] = Timer("map(lambda s: s.split(), a)", "a = ['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> t['listcomp'] = Timer("[s.split() for s in a]", "a = ['2011-12-22 46:31:11','2011-12-20 20:19:17', '2011-12-20 01:09:21']")
>>> for name, timer in t.items():
...     print '%s: %.2f usec/pass' % (name, 1000000 * timer.timeit(number=100000)/100000)
... 
listcomp: 2.08 usec/pass
methodcaller: 2.87 usec/pass
lambda: 3.10 usec/pass